In [1]:
import torch
import torch.nn as nn
import csv
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler

/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# 从CSV文件加载数据
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)  # 跳过文件头部
        for line in reader:
            sentence, label = line
            # 将标签转为整型，这里我们把'positive'转为1，把'negative'转为0
            label = 1 if label == 'positive' else 0
            data.append((sentence, label))
    return data

In [3]:
# 数据集类
class IMDbDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence, label = self.data[idx]
        encoding = self.tokenizer.encode_plus(
            sentence,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        return input_ids, attention_mask, torch.tensor(label)  # 标签转为Tensor类型

In [4]:
# 定义自定义模型
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_classes):
        super(LSTMClassifier, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.neck = nn.Linear(hidden_dim, 256)
        self.tail = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask):
        embedded = self.embedding(input_ids)
        _, (h_n, _) = self.lstm(embedded)
        features = self.neck(h_n[-1])
        features = self.dropout(features)
        logits = self.tail(features)
        return logits

In [5]:
# 定义阈值清洗方法
def threshold_cleaning(predictions, labels, threshold):
    clean_indices = []
    for i in range(len(predictions)):
        if abs(predictions[i] - labels[i]) <= threshold:
            clean_indices.append(i)
    return clean_indices

In [8]:
# 加载数据集并进行分割
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
data = load_data('IMDB-NY.csv')
train_data, val_data = train_test_split(data, test_size=0.2, random_state=4)

In [9]:
# 获取词汇表大小
vocab_size = len(tokenizer)

# 创建训练集和验证集
train_dataset = IMDbDataset(train_data, tokenizer)
val_dataset = IMDbDataset(val_data, tokenizer)

In [10]:
# 定义超参数
batch_size = 32
num_classes = 2
embedding_dim = 128
hidden_dim = 256
num_epochs = 50
threshold = 0.5

In [11]:
# 创建数据加载器
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [12]:
# 构建模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(embedding_dim, hidden_dim, num_classes).to(device)

In [13]:
# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [ ]:
# 训练模型
best_val_loss = float('inf')
no_improve_epoch = 0
max_no_improve_epoch = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    all_labels = []
    all_predictions = []

    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # 计算准确度
        predictions = torch.argmax(logits, dim=1)
        total_correct += (predictions == labels).sum().item()
        total_samples += labels.size(0)
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    average_loss = total_loss / len(train_dataloader)
    accuracy = total_correct / total_samples
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

    # 验证步骤
    model.eval()
    total_val_loss = 0
    total_val_correct = 0
    total_val_samples = 0
    all_val_labels = []
    all_val_predictions = []

    with torch.no_grad():
        for val_batch in val_dataloader:
            val_input_ids, val_attention_mask, val_labels = val_batch
            val_input_ids = val_input_ids.to(device)
            val_attention_mask = val_attention_mask.to(device)
            val_labels = val_labels.to(device)

            val_logits = model(val_input_ids, val_attention_mask)
            val_loss = criterion(val_logits, val_labels)

            total_val_loss += val_loss.item()

            # 计算验证集的准确度
            val_predictions = torch.argmax(val_logits, dim=1)
            total_val_correct += (val_predictions == val_labels).sum().item()
            total_val_samples += val_labels.size(0)
            all_val_predictions.extend(val_predictions.cpu().numpy())
            all_val_labels.extend(val_labels.cpu().numpy())

    average_val_loss = total_val_loss / len(val_dataloader)
    val_accuracy = total_val_correct / total_val_samples
    print(f'Val Loss: {average_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

    # 使用阈值清洗方法检测噪声
    clean_indices = threshold_cleaning(all_val_predictions, all_val_labels, threshold)
    clean_val_labels = np.array(all_val_labels)[clean_indices]
    clean_val_predictions = np.array(all_val_predictions)[clean_indices]

Epoch 1/50, Loss: 0.6932, Accuracy: 0.5074
Val Loss: 0.6932, Val Accuracy: 0.5094
Epoch 2/50, Loss: 0.6926, Accuracy: 0.5153
Val Loss: 0.6929, Val Accuracy: 0.5162
Epoch 3/50, Loss: 0.6921, Accuracy: 0.5187
Val Loss: 0.6925, Val Accuracy: 0.5233
Epoch 4/50, Loss: 0.6912, Accuracy: 0.5288
Val Loss: 0.6928, Val Accuracy: 0.5172
Epoch 5/50, Loss: 0.6900, Accuracy: 0.5352
Val Loss: 0.6921, Val Accuracy: 0.5263
Epoch 6/50, Loss: 0.6883, Accuracy: 0.5422
Val Loss: 0.6926, Val Accuracy: 0.5283
Epoch 7/50, Loss: 0.6858, Accuracy: 0.5498
Val Loss: 0.6906, Val Accuracy: 0.5344
Epoch 8/50, Loss: 0.6816, Accuracy: 0.5613
Val Loss: 0.6855, Val Accuracy: 0.5589
Epoch 9/50, Loss: 0.6688, Accuracy: 0.5869
Val Loss: 0.6714, Val Accuracy: 0.5839
Epoch 10/50, Loss: 0.6580, Accuracy: 0.6087
Val Loss: 0.6651, Val Accuracy: 0.5972
Epoch 11/50, Loss: 0.6451, Accuracy: 0.6325
Val Loss: 0.6561, Val Accuracy: 0.6173
Epoch 12/50, Loss: 0.6354, Accuracy: 0.6456
Val Loss: 0.6561, Val Accuracy: 0.6222
Epoch 13/50, 